# Classification Challenge using CatBoost

## INF2179 Fall 2021
### Hamid Yuksel

This submission uses [CatBoost](https://catboost.ai/).
CatBoost was chosen for its listed benefits, mainly in requiring less hyperparameter tuning and preprocessing of categorical and text features. It is also fast and fairly easy to set up.

<img src="https://cdn.britannica.com/39/7139-050-A88818BB/Himalayan-chocolate-point.jpg"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" />


In [119]:
#Installing and Importing required libraries
! pip3 install --user catboost
! pip3 install --user ipywidgets
! jupyter nbextension enable --py widgetsnbextension

import pandas as pd 
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score 
from catboost import Pool, CatBoostClassifier

You should consider upgrading via the '/Users/yuksel/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/yuksel/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [121]:
# Reading data
df = pd.read_csv('data.csv')

# Splitting
training = df.head(50000)
holdout_set = training.sample(5000, random_state=1) # pick 5000 observations randomly
training = training.drop(holdout_set.index) # Remove holdout from training data
testing = df.tail(5000)

#Looking at counts per genre
df['Genre'].value_counts()

Rock       24486
Pop        16251
Hip Hop     9263
unknown     5000
Name: Genre, dtype: int64

In [122]:
# Splitting training/testing set to feature (X) and labels (y)
train_y = training.Genre
train_X = training.drop('Genre', axis=1)

test_X = testing.drop('Genre', axis=1)

test_X

,Lyric
50000,"Feels so good,. Feels so good,. Feels so good ..."
50001,"Shadow of a doubt. I heard your heart,. you he..."
50002,Slaves. Hebrews born to serve to the pharaoh. ...
50003,You've been picked and it's over. What's the c...
50004,Magic happens. But only if you are open to the...
...,...
54995,I can't believe what you did to me. Down on my...
54996,Have all the songs been written?. Have all the...
54997,Everything you do you do so right. The clothes...
54998,(trecho). (Rule Number Two. Understanding what...


In [123]:
# Build a classifier
text_features = ['Lyric']


train_dataset = Pool(data=train_X,
                     label=train_y,
                     text_features=text_features)

model = CatBoostClassifier(iterations=100,
                           learning_rate=1,
                           depth=3,
                           loss_function='MultiClass')

model.fit(train_dataset)

0:	learn: 0.8096011	total: 81.1ms	remaining: 8.03s
1:	learn: 0.7791994	total: 150ms	remaining: 7.34s
2:	learn: 0.7603779	total: 224ms	remaining: 7.25s
3:	learn: 0.7547274	total: 290ms	remaining: 6.95s
4:	learn: 0.7483987	total: 367ms	remaining: 6.97s
5:	learn: 0.7445907	total: 432ms	remaining: 6.76s
6:	learn: 0.7401263	total: 501ms	remaining: 6.65s
7:	learn: 0.7385096	total: 578ms	remaining: 6.64s
8:	learn: 0.7358355	total: 654ms	remaining: 6.61s
9:	learn: 0.7336089	total: 757ms	remaining: 6.81s
10:	learn: 0.7320447	total: 816ms	remaining: 6.6s
11:	learn: 0.7291184	total: 882ms	remaining: 6.47s
12:	learn: 0.7262838	total: 959ms	remaining: 6.42s
13:	learn: 0.7228727	total: 1.04s	remaining: 6.37s
14:	learn: 0.7201308	total: 1.1s	remaining: 6.23s
15:	learn: 0.7178398	total: 1.19s	remaining: 6.26s
16:	learn: 0.7163848	total: 1.26s	remaining: 6.17s
17:	learn: 0.7146310	total: 1.33s	remaining: 6.06s
18:	learn: 0.7136866	total: 1.39s	remaining: 5.92s
19:	learn: 0.7126146	total: 1.47s	remainin

In [129]:
# Estimate accuracy
pred = model.predict(holdout_set.drop('Genre',axis=1))
estimated_accuracy = accuracy_score(holdout_set['Genre'], pred)
print(estimated_accuracy)
pd.Series(estimated_accuracy).to_csv('ea.csv', index=False, header=False)

0.6794


In [131]:
# Predict testing set
pred = model.predict(test_X)
print(pred.flatten())
pred = pd.Series(pred.flatten()).to_csv('pred.csv', index=False, header=False)

['Pop' 'Rock' 'Rock' ... 'Rock' 'Pop' 'Rock']


In [128]:
# to check number of instances of each genre in pred
np.unique(model.predict(test_X), return_counts=True)

(array(['Hip Hop', 'Pop', 'Rock'], dtype=object), array([ 802, 1307, 2891]))